# Output sample points from 5 different patients

In [1]:
basic_symptoms_file = "/shares/bulk/oagba/data/output_basic_15k/symptoms/csv/symptoms.csv"

In [2]:
from dask_jobqueue import SLURMCluster
import dask.dataframe as dd
from dask.distributed import Client

In [3]:
cluster = SLURMCluster(
    queue='general',
    # project='medvice_parse',
    cores=8,
    memory='34 GB',
    walltime='00:30:00',
    death_timeout=60
)

In [4]:
client = Client(cluster)

In [72]:
cluster.scale(2)

In [73]:
df = dd.read_csv(
    basic_symptoms_file,
)

In [6]:
cluster

SLURMCluster('tcp://131.180.106.142:42383', workers=2, threads=16, memory=68.00 GB)

In [126]:
appendicitis = "1eebbc48b667086fff2958b8419f68ac99bbcdb181c2ad835ece5abc"
acute_sinusitis= "4bb4bf3e5ddfd974958855fb6182ac3595e2047e9bbffcd1d81e41f7"
pharyngitis = "4fe54bc8061b567847f223d206732985923d29ee61e784c1a65bd5c8"

conditions = [appendicitis, acute_sinusitis, pharyngitis]

In [127]:
samples = [df[df.PATHOLOGY == item].sample(frac=0.05).head() for item in conditions]

In [128]:
_all_samples = pd.concat(samples)

In [21]:
condition_db_file = "/shares/bulk/oagba/data/definitions/condition_db.json"
symptom_db_file = "/shares/bulk/oagba/data/definitions/symptom_db.json"

with open(condition_db_file) as fp:
    condition_db = json.load(fp)
with open(symptom_db_file) as fp:
    symptom_db = json.load(fp)

In [27]:
def transform_symptoms(symptom_str, symptom_db):
    return ";".join([symptom_db.get(item) for item in symptom_str.split(";")])

def transform_condiition(condition, condition_db):
    return condition_db.get(condition)

In [19]:
op_directory = "/home/oagba/bulk/data/reports/samples"

In [32]:
import pandas as pd

In [123]:
_all_samples = pd.concat(samples)

In [129]:
_all_samples['SYMPTOMS'] = _all_samples.SYMPTOMS.apply(transform_symptoms, symptom_db=symptom_db)
_all_samples['PATHOLOGY'] = _all_samples.PATHOLOGY.apply(transform_condiition, condition_db=condition_db)

In [133]:
csv_file_op = os.path.join(op_directory, "samples.csv")
_all_samples.to_csv(csv_file_op, index=False)

In [134]:
len(symptom_db)

376